#**Handling Missing Data in ETL**

**SECTION A – THEORETICAL QUESTIONS **
Q1. What are the most common reasons for missing data in ETL pipelines?

Ans: The most common reasons for missing data in ETL are

1.Source System Issues

Data was never captured in the source (optional fields, user skipped input).

Manual entry errors (blank fields, incomplete forms).

2.Data Extraction Failures

Connection issues (API timeout, DB connection drop).

Incremental load logic missed records (wrong date filter, wrong key).

3.Schema Changes in Source

Source system adds/removes/renames columns.

ETL job still expects the old schema → data becomes NULL or ignored.

4.Data Transformation Errors

Incorrect joins (INNER JOIN instead of LEFT JOIN).

Filters remove rows unintentionally.

Data type conversion failures (string → date/number).

5.Data Quality Issues

Invalid values replaced with NULL during cleansing.

Business rules mark some values as missing.

6.Loading Issues

Constraint violations (NOT NULL, UNIQUE).

Records rejected during load but not logged properly.

7.Timing & Sync Problems

Source data not fully updated when ETL runs.

Late-arriving data.

8.Human / Configuration Errors

Wrong mapping in ETL tool.

Missing fields in source-to-target mapping.

Q2. Why is blindly deleting rows with missing values considered a bad practice in ETL?

Ans:The blindly deleting rows with missing values considered a bad practice in ETL are:

1.Data Loss

You may delete valuable and valid records.

Missing value in one column doesn’t mean the entire row is useless.

2.Bias in Analysis

Removing rows can distort trends and patterns.

Results may no longer represent real business behavior.

3.Reduced Data Volume

Large deletions reduce sample size → less reliable insights.
4.Business Impact

Missing data might be meaningful (e.g., NULL salary = not disclosed).

Deleting such rows can misrepresent business rules.

5.Hidden Upstream Issues

Deleting data hides source system or ETL problems.

Root causes never get fixed.

6.Compliance & Audit Risks

Some industries require data traceability.

Deletion may violate audit or regulatory rules.

7.Future Use Cases Break

Data that seems useless today might be useful later.

Once deleted, it’s gone forever.

Q3. Explain the difference between:

Listwise deletion

Column deletion

Ans:The difference between listwise deletion and column deletion are

Aspect	Listwise Deletion	Column Deletion
Meaning	Deletes the entire row if any column has a missing value	Deletes the entire column if it has many missing values
What is removed	Rows	Columns
Data loss	High (row-level data lost)	Attribute-level data loss
Bias risk	High	Lower
Impact on dataset	Reduces number of records	Reduces number of features
Appropriate scenario	When strict completeness is required and missing data is very small (e.g., financial or regulatory reports)	When a column has very high missing % and low business importance (e.g., fax number)
ETL usage	Rare in production pipelines	Sometimes used
Example	Removing records with any NULL in a compliance dataset	Dropping a legacy or unused column
Q4. Why is median imputation preferred over mean imputation for skewed data such as income?

Ans:The median imputation preferred over mean imputation for skewed data such as income are

Reason	Explanation
Robust to outliers	Income data usually has extreme high values. The mean gets pulled up, but the median stays stable.
Represents typical value	The median reflects the central tendency of skewed data better than the mean.
Reduces distortion	Mean imputation can inflate or deflate values unrealistically in skewed distributions.
Preserves distribution shape	Median imputation keeps the original skewness more intact.
Better business sense	Most people earn around the median income, not the average influenced by top earners.
Q5. What is forward fill and in what type of dataset is it most useful?

Ans:Forward fill means replacing a missing value with the last available (previous) non-missing value in the dataset.

Typical datasets include:

Stock prices

Sensor readings (IoT data)

Weather data

Daily sales or inventory levels

Log or monitoring data

Q6. Why should flagging missing values be done before imputation in an ETL workflow?

Ans:We should flagging missing values be done before imputation in an ETL workflow are

Reason	Explanation
Preserves original information	Flagging keeps a record of which values were originally missing before they were filled.
Enables better analysis	Analysts and models can treat imputed values differently from real values.
Improves transparency	Makes the ETL process auditable and explainable.
Helps detect data quality issues	High flags indicate upstream source problems.
Prevents misleading insights	Imputed values look real unless flagged, which can bias reports and models.
Supports future reprocessing	Raw missingness info allows re-imputation with better methods later.
Q7. Consider a scenario where income is missing for many customers. How can this missingness itself provide busi

Ans:Missing income data can itself be informative and not just a data quality issue.

Business insights from missing income:

Customers with missing income may belong to informal sectors, freelancers, students, or unemployed groups.

They might be less willing to disclose financial details, indicating privacy concerns or low trust.

Missing income could correlate with lower creditworthiness, higher risk, or different spending behavior.

In some cases, high-net-worth customers also skip income fields, assuming it’s irrelevant, which signals a premium segment.

The pattern of missing income across regions, age groups, or products can help businesses refine targeting, pricing, and risk models.

SECTION B – PRACTICAL QUESTIONS
given datasets

Customer_ID	Name	City	Monthly_Sales	Income	Region
101	Rahul Mehta	Mumbai	12000	65000	West
102	Anjali Roy	Bengaluru	NaN	50000	South
103	Suresh Iyer	Chennai	15000	72000	South
104	Nikhil Shah	Delhi	NaN	NaN	North
105	Amit Verma	Pune	18000	58000	NaN
106	Karan Shah	Ahmedabad	NaN	60000	West
107	Pooja Das	Kolkata	14000	NaN	East
108	Riya Kapoor	Jaipur	16000	69000	North
Q8. Listwise Deletion

Remove all rows where Region is missing

Tasks:

1.Identify affected rows

Ans:Customer 105 (Amit Verma) has Region = NaN

2.Dataset after deletion

Customer_ID	Name	City	Monthly_Sales	Income	Region
101	Rahul Mehta	Mumbai	12000	65000	West
102	Anjali Roy	Bengaluru	NaN	50000	South
103	Suresh Iyer	Chennai	15000	72000	South
104	Nikhil Shah	Delhi	NaN	NaN	North
106	Karan Shah	Ahmedabad	NaN	60000	West
107	Pooja Das	Kolkata	14000	NaN	East
108	Riya Kapoor	Jaipur	16000	69000	North
3.How many records were lost?

1 record deleted
Q9. Imputation

Handle missing values in Monthly_Sales using

Forward Fill

Missing Monthly_Sales rows:

Customer 102

Customer 104

Customer 106

Tasks

1.Apply Forward Fill

Customer_ID	Monthly_Sales	Filled Value
102	NaN	12000
104	NaN	15000
106	NaN	18000
2.Before vs After

Before:

12000 → NaN → 15000 → NaN → 18000 → NaN → 14000 → 16000
After (Forward Fill):

12000 → 12000 → 15000 → 15000 → 18000 → 18000 → 14000 → 16000
3.Explain why forward fill is suitable here

Ans:We use forward fill is suitable here are

Monthly sales often follow recent trends

Prevents sudden unrealistic jumps

Maintains time/order-based continuity

Simple and effective when data is sequential

Q10. Flagging Missing Data

Create a flag column for missing Income

1.Create Income_Missing_Flag

0 = Income present

1 = Income missing

Customer_ID	Income	Income_Missing_Flag
101	65000	0
102	50000	0
103	72000	0
104	NaN	1
105	58000	0
106	60000	0
107	NaN	1
108	69000	0
Q2.Show updated dataset?

Ans: The updated datasets are

Customer_ID	Name	City	Monthly_Sales	Income	Region	Income_Missing_Flag
101	Rahul Mehta	Mumbai	12000	65000	West	0
102	Anjali Roy	Bengaluru	NaN	50000	South	0
103	Suresh Iyer	Chennai	15000	72000	South	0
104	Nikhil Shah	Delhi	NaN	NaN	North	1
105	Amit Verma	Pune	18000	58000	NaN	0
106	Karan Shah	Ahmedabad	NaN	60000	West	0
107	Pooja Das	Kolkata	14000	NaN	East	1
108	Riya Kapoor	Jaipur	16000	69000	North	0
Task 3: Count of customers with missing income

Customers with missing income: 104, 107

Total customers with missing income = 2